# Spam data

In [ ]:
import numpy as np
import pandas as pd
import glob

In [ ]:
path = r'data/text'
allFiles = glob.glob(path+'/*.csv')

### Get files and append them into DataFrame

In [ ]:
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col = None, header=0)
    list_.append(df)
frame = pd.concat(list_)
print(df.head())
df = frame.iloc[:,3:5]
print(df.head())

### Split data to training and testing!

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

x = df['CONTENT']

y = df['CLASS']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=53)

In [ ]:
print('train size={} test size={} full data size={}'.format(x_train.shape, y_test.shape, x.shape))

### Generate features from training and testing data

In [ ]:
# - create features using countvectorizer
count_vectorizer = CountVectorizer(stop_words='english')

# creates the document term matrix
count_train = count_vectorizer.fit_transform(x_train.values)

count_test = count_vectorizer.transform(x_test.values)

### Train the ML model for a given sensitivity value for naive bayesian classifier

In [ ]:
# create Naive Bayes classifier with smoothing parameter
nb_classifier = MultinomialNB(alpha=0.1)

#fit the classifier to training data
nb_classifier.fit(count_train, y_train)

pred = nb_classifier.predict(count_test)
score = metrics.accuracy_score(y_test, pred)

### show some predictions

In [ ]:
results = pd.DataFrame({'comment':x_test[:-1].values, 'label':pred})
print(results.head())